**Lo que realice a continuacion es el conjunto de las ultimas 4 entregas ya que me di cuenta que habia mal interprtado las consignas y entendi que habia que realizarlas con un dataset nuevo o distinto al que veniamos utilizando. Por lo tanto a la hora de realizar la presentacion del proyecto final realice los ultimos 4 desafios juntos en esta notebook para hacerlo de forma mas rapida y que me permita continuar con la presentacion del proyecto**

# Preparación del entorno

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer

Reemplace el link de drive por el link del csv de github

In [5]:
# Cargue el conjunto de datos en un DataFrame de Pandas
df = pd.read_csv('https://raw.githubusercontent.com/Bruno-Leguiza/Proyecto-Final-de-Data-Science---Coderhouse/main/train.csv')

# **Encoding**

# Identificacion de Variables Categoricas

In [6]:
# Visualización de información del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [7]:
# Identificación de variables categóricas
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
print(categorical_cols)

['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


## Aplicamos Técnicas de Encoding:

In [8]:
# Aplicamos One-Hot Encoding a las variables categóricas
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# **Ingeniería de Atributos**

# Creación de Nuevas Características

**Creamos una nueva característica basada en la antigüedad de la propiedad:**

Esta nueva característica es la diferencia entre el año de venta y el año de construcción o remodelación.

In [9]:
# Creamos nueva característica 'AntigüedadPropiedad'
df_encoded['AntigüedadPropiedad'] = df_encoded['YrSold'] - df_encoded['YearBuilt']

**Agregamos el área total del sótano:**

Sumamos las áreas de los diferentes tipos de áreas del sótano para obtener un área total de sótano.

In [10]:
# Creamos nueva característica 'AreaTotalSotano'
df_encoded['AreaTotalSotano'] = df_encoded['BsmtFinSF1'] + df_encoded['BsmtFinSF2'] + df_encoded['BsmtUnfSF']

**Combinamos características relacionadas:**

Combinamos características relacionadas para formar una nueva característica más significativa.

In [11]:
# Combinar 'TotalBaños' sumando baños completos y medio baños
df_encoded['TotalBaños'] = df_encoded['FullBath'] + df_encoded['HalfBath']

**Eliminación de Características Redundantes**

Eliminamos características redundantes o poco informativas para mejorar el rendimiento del modelo.

In [12]:
columns_to_drop = ['Id', 'YearBuilt', 'YrSold', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'FullBath', 'HalfBath']
df_encoded = df_encoded.drop(columns=columns_to_drop, axis=1)

# **Entrenamiento del Modelo de Machine Learning**

Para este caso, dado que estamos buscando predecir el precio de las viviendas (una variable continua), utilizaremos un modelo de regresión lineal

In [13]:
# Separamos las características (variables independientes) y la variable objetivo
X = df_encoded.drop('SalePrice', axis=1)  # Características
y = df_encoded['SalePrice']  # Variable objetivo

In [14]:
# Dividimos el conjunto de datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Crear un imputador para llenar los valores faltantes con la media
imputer = SimpleImputer(strategy='mean')

# Ajustar el imputador a los datos de entrenamiento y transformar los conjuntos de entrenamiento y prueba
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

In [16]:
# Creamos una instancia del modelo de regresión lineal
model = LinearRegression()

In [17]:
# Entrenar el modelo con los datos de entrenamiento imputados
model.fit(X_train_imputed, y_train)

LinearRegression()

In [18]:
# Predicciones con el conjunto de prueba
predictions = model.predict(X_test_imputed)

Para evaluar la calidad del modelo y su capacidad para predecir los precios de las viviendas, podremos utilizar métricas de evaluación de regresión como el error cuadrático medio (Mean Squared Error - MSE) y el coeficiente de determinación (R-cuadrado)

In [19]:
from sklearn.metrics import mean_squared_error, r2_score

# Calculamos el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error (MSE): {mse}")

# Calculamos el coeficiente de determinación (R-cuadrado)
r2 = r2_score(y_test, predictions)
print(f"R-squared (R2): {r2}")

Mean Squared Error (MSE): 2217301088.8489113
R-squared (R2): 0.7109247991602636


# Interpretación de estos valores:

El MSE es una medida del promedio de los cuadrados de los errores, por lo que un MSE más bajo indica un mejor ajuste del modelo a los datos. En este caso, el MSE es relativamente alto, lo que quiere decir que el modelo todavía tiene cierto margen de mejora en cuanto a ajuste de los datos de prueba.

El coeficiente de determinación (R-cuadrado) es una medida que indica la proporción de la varianza en la variable dependiente que es predecible a partir de las variables independientes. Un valor de R-cuadrado cercano a 1 indica un mejor ajuste del modelo a los datos. En este caso, un R-cuadrado de aproximadamente 0.711 indica que el modelo explica aproximadamente el 71.1% de la varianza de los datos de prueba, por lo tanto es un buen resultado, pero aún deja margen para mejoras.

# **Nueva ingeniería de atributos**

# Agregación de características relacionadas:
Para esta parte, estaremos creando una nueva característica que capture el total de áreas exteriores, sumando las áreas de porches y espacios exteriores.
Esto ayudará a consolidar la información sobre las áreas exteriores, proporcionando una única característica que refleje la suma total de estas áreas.

In [20]:
# CreaMOS una nueva característica para capturar el total de áreas exteriores
df_encoded['TotalAreasExteriores'] = df_encoded['WoodDeckSF'] + df_encoded['OpenPorchSF'] + df_encoded['EnclosedPorch'] + df_encoded['3SsnPorch'] + df_encoded['ScreenPorch']

# Interacciones entre características

En esta sección, realizaremos la multiplicación de características para crear interacciones que puedan tener más significado en la predicción. Esta interacción podría capturar una relación potencial entre la calidad general de la casa y el área habitable, lo cual puede ser relevante en la predicción del precio.

In [21]:
# Multiplicar características para crear interacciones
# La interacción entre 'OverallQual' y 'GrLivArea'
df_encoded['OverallQual_GrLivArea'] = df_encoded['OverallQual'] * df_encoded['GrLivArea']

# Transformaciones adicionales:
Para abordar posibles diferencias en escalas entre características, aplicaremos el escalado de características utilizando MinMaxScaler de sklearn. Escalaremos las características seleccionadas para que estén en la misma escala, lo que puede mejorar el rendimiento del modelo.

In [22]:
from sklearn.preprocessing import MinMaxScaler

# Escalar características si tienen diferentes escalas
scaler = MinMaxScaler()
columns_to_scale = ['LotArea', 'TotalBsmtSF', '1stFlrSF', 'GrLivArea', 'GarageArea']
df_encoded[columns_to_scale] = scaler.fit_transform(df_encoded[columns_to_scale])

# Manejo avanzado de valores faltantes:
Para el manejo avanzado de valores faltantes, utilizaremos IterativeImputer de sklearn para imputar los valores faltantes utilizando un modelo predictivo.

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Imputamos valores faltantes utilizando IterativeImputer
imputer_advanced = IterativeImputer(random_state=42)
df_encoded_imputed = imputer_advanced.fit_transform(df_encoded)
df_encoded_imputed = pd.DataFrame(df_encoded_imputed, columns=df_encoded.columns)

El uso de **IterativeImputer** permite llenar los valores faltantes de manera más avanzada, utilizando información de las otras características para realizar la imputación.

# **Entrenamiento del Modelo de Machine Learning**

In [ ]:
# División del conjunto de datos
X = df_encoded_imputed.drop('SalePrice', axis=1)  # Características
y = df_encoded_imputed['SalePrice']  # Variable objetivo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Entrenamiento de un modelo de regresión
# Aquí estamos usando nuevamente LinearRegression como ejemplo, pero podrías explorar otros modelos como RandomForestRegressor, GradientBoostingRegressor, etc.
model_improved = LinearRegression()
model_improved.fit(X_train, y_train)

In [ ]:
# Predicciones con el conjunto de prueba utilizando el modelo mejorado
predictions_improved = model_improved.predict(X_test)

# Evaluación del modelo mejorado
mse_improved = mean_squared_error(y_test, predictions_improved)
r2_improved = r2_score(y_test, predictions_improved)

# Resultados del modelo mejorado
print(f"Mean Squared Error (MSE) del modelo mejorado: {mse_improved}")
print(f"R-squared (R2) del modelo mejorado: {r2_improved}")

# **Implementación de Validación Cruzada**

# Manejo de datos faltantes:
Antes de aplicar K-fold Cross Validation, verificamos si quedan valores faltantes en el conjunto de datos y realizamos una imputación, si es necesario.

In [ ]:
# Verificamos valores faltantes
missing_values = df_encoded_imputed.isnull().sum()
print("Valores faltantes por columna:\n", missing_values)

# Revisión de la relevancia de las características:
Vamos a asegurarnos de utilizar solo las características más relevantes para predecir el precio de venta. Emplearemos técnicas de selección de características como la importancia de las características o análisis univariado.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# Seleccionamos las características más relevantes
selector = SelectKBest(score_func=f_regression, k=10)  # Selecciona las 10 mejores características
X_selected = selector.fit_transform(X, y)

# Obtenemos los índices de las características seleccionadas
selected_indices = selector.get_support(indices=True)
selected_features = X.columns[selected_indices]
print("Características más relevantes:\n", selected_features)

# Aplicamos K-fold Cross Validation:
Aplicaremos K-fold Cross Validation para evaluar el rendimiento del modelo. Usaremos el modelo de regresión lineal

In [ ]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
import numpy as np

# Definimos el modelo de regresión lineal
model = LinearRegression()

# Definimos K-fold Cross Validation con k = 5
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

# Realizamos la validación cruzada
mse_scores = cross_val_score(model, X[selected_features], y, scoring='neg_mean_squared_error', cv=k_fold)

# Calculamos el promedio del MSE obtenido en las distintas iteraciones
avg_mse = np.mean(-mse_scores)  # Se multiplica por -1 porque cross_val_score devuelve el negativo del MSE

print("Promedio del MSE utilizando K-fold Cross Validation:", avg_mse)

**Lo que esta debajo de este texto fue realizado de forma rapida mientras desarrollaba la presentacion del proyecto final, ya que anteriormente habia desarrollado un unico modelo de machine learning que era el modelo de regresion lineal, al estar haciendo la presentacion del proyecto me di cuenta que tenia que probar con otros modelos para compara resultados y eso me dio exito. Se ve reflejado en los resultados y esta documentado en la presentacion del proyecto**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Creamos una instancia del modelo RandomForestRegressor
rf_model = RandomForestRegressor(random_state=42)

# Entrenamos el modelo con los datos de entrenamiento
rf_model.fit(X_train_imputed, y_train)

# Realizamos predicciones con el conjunto de prueba
rf_predictions = rf_model.predict(X_test_imputed)

# Evaluamos el rendimiento del modelo
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_r2 = r2_score(y_test, rf_predictions)

print(f"RandomForestRegressor - Mean Squared Error (MSE): {rf_mse}")
print(f"RandomForestRegressor - R-squared (R2): {rf_r2}")

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Creamos una instancia del modelo GradientBoostingRegressor
gb_model = GradientBoostingRegressor(random_state=42)

# Entrenamos el modelo con los datos de entrenamiento
gb_model.fit(X_train_imputed, y_train)

# Realizamos predicciones con el conjunto de prueba
gb_predictions = gb_model.predict(X_test_imputed)

# Evaluamos el rendimiento del modelo
gb_mse = mean_squared_error(y_test, gb_predictions)
gb_r2 = r2_score(y_test, gb_predictions)

print(f"GradientBoostingRegressor - Mean Squared Error (MSE): {gb_mse}")
print(f"GradientBoostingRegressor - R-squared (R2): {gb_r2}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Definimos el modelo
rf = RandomForestRegressor()

# Definimos el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Aplicamos GridSearchCV para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Muestra los mejores hiperparámetros encontrados
print(grid_search.best_params_)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Definimos el modelo
gb = GradientBoostingRegressor()

# Definimos el espacio de búsqueda de hiperparámetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7]
}

# Aplicamos GridSearchCV para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=gb, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search.fit(X_train, y_train)

# Muestra los mejores hiperparámetros encontrados
print(grid_search.best_params_)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Definimos el modelo con los mejores hiperparámetros encontrados
rf_model_optimized = RandomForestRegressor(max_depth=30, min_samples_split=5, n_estimators=200, random_state=42)

# Entrenamos el modelo con los datos de entrenamiento
rf_model_optimized.fit(X_train, y_train)

# Realizamos predicciones con el conjunto de prueba
rf_predictions_optimized = rf_model_optimized.predict(X_test)

# Evaluamos el rendimiento del modelo optimizado
rf_mse_optimized = mean_squared_error(y_test, rf_predictions_optimized)
rf_r2_optimized = r2_score(y_test, rf_predictions_optimized)

print(f"RandomForestRegressor Optimizado - Mean Squared Error (MSE): {rf_mse_optimized}")
print(f"RandomForestRegressor Optimizado - R-squared (R2): {rf_r2_optimized}")

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Definimos el modelo con los mejores hiperparámetros encontrados
gb_model_optimized = GradientBoostingRegressor(learning_rate=0.1, max_depth=3, n_estimators=300, random_state=42)

# Entrenamos el modelo con los datos de entrenamiento
gb_model_optimized.fit(X_train, y_train)

# Realizamos predicciones con el conjunto de prueba
gb_predictions_optimized = gb_model_optimized.predict(X_test)

# Evaluamos el rendimiento del modelo optimizado
gb_mse_optimized = mean_squared_error(y_test, gb_predictions_optimized)
gb_r2_optimized = r2_score(y_test, gb_predictions_optimized)

print(f"GradientBoostingRegressor Optimizado - Mean Squared Error (MSE): {gb_mse_optimized}")
print(f"GradientBoostingRegressor Optimizado - R-squared (R2): {gb_r2_optimized}")